In [ ]:
# Initial imports
import numpy as np
import pandas as pd
import os
import sqlite3
import gzip
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
'''
# Define the paths to form the QSL database
base_path = "./" 
folders = ['hosp', 'icu', 'note']
db_path = "mimic.db"

# Connect to SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Iterate through each folder and process .csv.gz files
for folder in folders: 
    folder_path = os.path.join(base_path, folder)
    
    for file in os.listdir(folder_path):
        if file.endswith(".csv.gz"):
            file_path = os.path.join(folder_path, file)
            table_name = file.replace(".csv.gz", "")

            chunk_size = 1000000
            for chunk in pd.read_csv(file_path, compression='gzip', chunksize= chunk_size):
                chunk.to_sql(table_name, conn, if_exists='append', index=False)
            
            print(f'Finished processing {file} into table {table_name}')

# Close the connection
conn.close()
'''

In [ ]:
db_path= '../database/mimic.db'

In [ ]:
# Connect to the SQLite database to query the files
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [ ]:
# Patients
patients = pd.read_sql_query("""SELECT subject_id, gender, anchor_age FROM patients""", conn)

# LAdmissions - filtered by patients' number of admissions
admissions = pd.read_sql_query("""SELECT subject_id, hadm_id, admittime, dischtime FROM admissions WHERE subject_id IN (SELECT subject_id FROM admissions GROUP BY subject_id HAVING COUNT(hadm_id) = 2) ORDER BY subject_id""", conn)


In [ ]:
# Merge admissions with patients
admissions_patients = admissions.merge(patients, on= 'subject_id', how='left')
display(admissions_patients.head())
print(admissions_patients['subject_id'].nunique())

In [ ]:
# Load the d_icd_diagnoses into a database
query_d_icd_diagnoses = """SELECT * FROM d_icd_diagnoses;"""
d_icd_diagnoses_df = pd.read_sql_query(query_d_icd_diagnoses, conn)

# Load the diagnoses_icd into a database
query_diagnoses_icd = "SELECT * FROM diagnoses_icd;"
diagnoses_icd_df = pd.read_sql_query(query_diagnoses_icd, conn)

# 1. Merge the two databases into a diagnoses database
diagnoses = diagnoses_icd_df.merge(d_icd_diagnoses_df, on=['icd_code', 'icd_version'], how = 'inner')

# 2. Count the frequent diagnoses
frequent_diagnoses = (
    diagnoses_icd_df.groupby(['icd_code', 'icd_version'], as_index=False)
    .size()
    .rename(columns={'size': 'count'})
)
frequent_diagnoses = frequent_diagnoses[frequent_diagnoses['count'] >= 10000]   # filter frequent diagnoses

# 3. Filter the diagnoses table with the frequent diagnoses
diagnoses_df = diagnoses.merge(frequent_diagnoses[['icd_code', 'icd_version']], on=['icd_code', 'icd_version'], how = 'inner')
display(diagnoses_df.head())

In [ ]:
# Define the SQL query
'''query = """
SELECT COUNT(*) AS total_drugs_with_multiple_units
FROM (
    SELECT drug
    FROM prescriptions
    GROUP BY drug
    HAVING COUNT(DISTINCT LOWER(dose_unit_rx)) > 1
);
"""
# Execute the query and fetch the results
cursor.execute(query)
result = cursor.fetchone()

# Print the total number of drugs with multiple dose units
print(f"Total drugs with multiple dose units: {result[0]}")
'''

In [ ]:
# Load the prescriptions table into a dataframe
query_prescriptions = """SELECT subject_id, hadm_id, drug, dose_val_rx, dose_unit_rx FROM prescriptions;"""
prescriptions = pd.read_sql_query(query_prescriptions, conn)

In [ ]:
# Convert "drug" and "dose_unit_rx" into lowercase
prescriptions['drug'] = prescriptions['drug'].str.lower()
prescriptions['dose_unit_rx'] = prescriptions['dose_unit_rx'].str.lower()

# Filter drugs with consistent drug units
valid_drugs = prescriptions.groupby('drug')['dose_unit_rx'].nunique()
consistent_drugs = valid_drugs[valid_drugs == 1].index  # drugs with only one single unit
consistent_presc_df = prescriptions[prescriptions['drug'].isin(consistent_drugs)]  # filter the dataset

# Remove the low-frequency drugs 
drug_counts = consistent_presc_df['drug'].value_counts()
frequent_drugs = drug_counts[drug_counts >= 1000].index  # Only drugs with at least 1000 occurrences

# final dataset
prescriptions_df = consistent_presc_df[consistent_presc_df['drug'].isin(frequent_drugs)]
prescriptions_df.head()

In [ ]:
conn.close()

In [ ]:
adm_df = admissions_patients.copy()
diag_df = diagnoses_df.copy()
prescr_df = prescriptions_df.copy()

In [ ]:
# Calculate length of stay and add the number of stay
adm_df['length_of_stay'] = (pd.to_datetime(adm_df['dischtime']) - pd.to_datetime(adm_df['admittime'])).dt.days
adm_df['stay'] = adm_df.groupby('subject_id')['hadm_id'].transform(lambda x: x.rank(method='dense'))
adm_df.head(4)

In [ ]:
# Column for first LOS 
first_stay_lengths = adm_df[adm_df['stay'] == 1].groupby('subject_id')['length_of_stay'].first().reset_index()
first_stay_lengths = first_stay_lengths.rename(columns={'length_of_stay': 'lengths_of_1st_admission'})

# Column for second LOS
second_stay_lengths = adm_df[adm_df['stay'] == 2].groupby('subject_id')['length_of_stay'].first().reset_index()
second_stay_lengths = second_stay_lengths.rename(columns={'length_of_stay': 'lengths_of_2nd_admission'})

# Merge 
stays = first_stay_lengths.merge(second_stay_lengths, on='subject_id')
display(first_stay_lengths.head())
display(second_stay_lengths.head())
display(stays.head())

In [ ]:
# Merge the stays with the admissions dataframe
adm_df = adm_df.merge(stays, on='subject_id')
adm_df = adm_df.drop(columns =['length_of_stay', 'stay'])
adm_df.head(4)

In [ ]:
# Set the target variable "lengths_of_2nd_admission" as binary for classification
adm_df['lengths_of_2nd_admission'] = (adm_df['lengths_of_2nd_admission'] >= 3).astype(int)

In [ ]:
# One-hot encode diagnoses per subject_id
diagnosis_pivot = diag_df.pivot_table(index='subject_id', columns='long_title', aggfunc='size', fill_value=0)
diagnosis_pivot = (diagnosis_pivot > 0).astype(int)  # set as binary values
diagnosis_pivot.reset_index(inplace=True)
diagnosis_pivot.head()

In [ ]:
# Ensure the dose column is numeric
prescr_df['dose_val_rx'] = pd.to_numeric(prescr_df['dose_val_rx'], errors='coerce')

In [ ]:
# select and filter the 68 medications
top_medications = prescr_df['drug'].value_counts().head(68).index
filtered_med_df = prescr_df[prescr_df['drug'].isin(top_medications)]

In [ ]:
# Pivot table for sum of doses per drug
sum_dose_pivot = filtered_med_df.pivot_table(index='subject_id', columns='drug', values='dose_val_rx', aggfunc='sum', fill_value=0)
sum_dose_pivot.columns = [f'{col} Sum' for col in sum_dose_pivot.columns]  # Rename columns

# Pivot table for mean dose per drug
avg_dose_pivot = filtered_med_df.pivot_table(index='subject_id', columns='drug', values='dose_val_rx', aggfunc='mean', fill_value=0)
avg_dose_pivot.columns = [f'{col} Average' for col in avg_dose_pivot.columns]  # Rename columns

display(sum_dose_pivot.head())
display(avg_dose_pivot.head())

In [ ]:
# Merge the two dose doses datasets
med_pivot = sum_dose_pivot.merge(avg_dose_pivot, on='subject_id', how='left')
med_pivot.reset_index(inplace=True)
med_pivot.head()

In [ ]:
# Merge admissions with prescriptions
final_df = adm_df.merge(med_pivot, on = ['subject_id'], how='inner')
display(final_df.head(3))
print(final_df['subject_id'].nunique())

In [ ]:
# Merge dataset with diagnoses
final_df = final_df.merge(diagnosis_pivot, on = ['subject_id'], how='inner')
display(final_df.head(3))
print(final_df['subject_id'].nunique())

In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [ ]:
# Create a temporary table from final_df for subject_id and hadm_id
final_df[['subject_id', 'hadm_id', 'admittime']].to_sql('temp_los', conn, if_exists='replace', index=False)

In [ ]:
# Get the discharge notes from the discharge table ONLY for the first admisison
discharge_text = pd.read_sql_query("""
    SELECT d.subject_id, d.hadm_id, d.text
    FROM discharge d
    JOIN (
        SELECT subject_id, hadm_id
        FROM temp_los
        WHERE admittime = (SELECT MIN(admittime) 
                           FROM temp_los t 
                           WHERE t.subject_id = temp_los.subject_id)
    ) first_admissions 
    ON d.subject_id = first_admissions.subject_id 
    AND d.hadm_id = first_admissions.hadm_id
""", conn)
discharge_text.drop(columns='hadm_id', inplace=True)

In [ ]:
# Merge discharge note text into final_df 
final_df = final_df.merge(discharge_text, on=['subject_id'], how='left')
final_df.head(2)

In [ ]:
adm_type = pd.read_sql_query("""
    SELECT subject_id, hadm_id,
           admission_type 
    FROM admissions
    GROUP BY subject_id, hadm_id
""", conn)


In [ ]:
# Merge the DRG data into final_df
admtype_pivot =  adm_type.pivot_table(index='subject_id', columns = 'admission_type', aggfunc='size', fill_value=0)
admtype_pivot = (admtype_pivot > 0).astype(int)
final_df = final_df.merge(admtype_pivot, on=['subject_id'], how='left')


In [ ]:
drg_sums = pd.read_sql_query("""
    SELECT subject_id, hadm_id,
           SUM(drg_severity) AS sum_drg_severity,
           SUM(drg_mortality) AS sum_drg_mortality
    FROM drgcodes
    GROUP BY subject_id, hadm_id
""", conn)


In [ ]:
# Create a df with the DRG values for the admission dates
first_hadm = final_df.groupby('subject_id')['hadm_id'].first().reset_index()
drg_sums.drop(columns='subject_id', inplace=True)
merged_severity = first_hadm.merge(drg_sums, on='hadm_id', how='left')
merged_severity.head(2)


In [ ]:
# Merge the values with the final dataset
final_df = final_df.merge(merged_severity[['subject_id', 'sum_drg_severity', 'sum_drg_mortality']], on='subject_id', how='left')

# fill null values with 0 (scores from 1 to 4)
final_df['sum_drg_severity'] = final_df['sum_drg_severity'].fillna(0)
final_df['sum_drg_mortality'] = final_df['sum_drg_mortality'].fillna(0)

In [ ]:
input_weights = pd.read_sql_query("""
    SELECT i.subject_id, i.hadm_id, i.patientweight
    FROM inputevents i
""", conn)

In [ ]:
# Calculate the average standard deviation for weight across all patients
std_per_patient = input_weights.groupby('subject_id')['patientweight'].std()
average_std = std_per_patient.mean()
average_std

In [ ]:
# Aggregate the dataset to have only one row per subject_id
df_aggregated = input_weights.groupby('subject_id')["patientweight"].mean().reset_index()

# Now you can merge it with your other dataset
final_df = final_df.merge(df_aggregated, on='subject_id', how='left')
final_df.head(2)

In [ ]:
# Integrate with the weight from omr table
omr_weights = pd.read_sql_query("""
    SELECT o.subject_id, o.result_value AS patientweight_omr
    FROM omr o
    WHERE o.result_name = 'Weight (Lbs)'
""", conn)
omr_weights.head()

In [ ]:
omr_weights['patientweight_omr'] = pd.to_numeric(omr_weights['patientweight_omr'], errors='coerce')

In [ ]:
# Calculate the average standard deviation for weight across all patients
std_per_patient_omr = omr_weights.groupby('subject_id')['patientweight_omr'].std()
average_std_omr = std_per_patient_omr.mean()
average_std_omr

In [ ]:
unique_subjects = final_df[['subject_id']].drop_duplicates()
merged_weights = unique_subjects.merge(omr_weights, on='subject_id', how='left')
merged_weights.head(2)

In [ ]:
# Calculate the average standard deviation for weight across all patients
std_per_patient_omr = merged_weights.groupby('subject_id')['patientweight_omr'].std()
average_std_omr = std_per_patient_omr.mean()
average_std_omr

In [ ]:
# Aggregate the dataset to have only one row per subject_id
avg_weight = omr_weights.groupby('subject_id', as_index=False)['patientweight_omr'].mean()
avg_weight.head()

In [ ]:
# Now you can merge it with your other dataset
final_df = final_df.merge(avg_weight, on='subject_id', how='left')
final_df.head(2)

In [ ]:
# Combine the two weight sources: use omr weight if available
final_df['patientweight'] = final_df['patientweight_omr'].combine_first(final_df['patientweight'])


In [ ]:
# Drop the helper column
final_df.drop(columns=['patientweight_omr'], inplace=True)

In [ ]:
# Closing connection
conn.close()

In [ ]:
# Remove the column hadm_id from admissions table and drop duplicated rows
final_df.drop(columns=['hadm_id', 'admittime', 'dischtime'], inplace=True)
final_df.drop_duplicates(inplace=True)